In [ ]:
# 流れ
# ・成果シートのデータ全取得
# ・必要なデータ列に絞る
# ・要らない行削る
# ・ASPカラム付け加える
# ・各ASPで上記行い合算する
# ・日付でソート
# ・今月のシートに反映

In [ ]:
# 成果確認シートの内容取得

In [ ]:
####################
# 
# モジュール
# 
####################

# 一般
import datetime
import pandas as pd

# スプシ関連
from gspread_dataframe import set_with_dataframe

# 自作
import conf
import func

print("モジュールインポート")

In [ ]:
####################
# 
# 変数
# 
####################

# 今月の成果シート名
today = datetime.date.today()
this_year  = str(today.year)
this_month = str(today.month)
sheet_name_seika = this_year + "年" + this_month + "月成果" 

print("変数定義完了")

In [ ]:
####################
# 
# 成果シートの情報取得
# 
####################

In [76]:
# 成果シートにアクセス
key_seika = "1tYvW0ntpbCVOpY2UGcqdQD_DQhA7HesfKkg0NPAp65g" # ブリーチ
wb_seika = func.connect_gspread(key_seika)
ws_seika = wb_seika.worksheet(sheet_name_seika)

# 成果シートのデータ取得
df_seika = pd.DataFrame(ws_seika.get_all_values()) # 元データ取得
df_seika = df_seika.drop(index=df_seika.index[[0,1]]) # 先頭行削除
df_seika = df_seika.reindex(columns=[9,1,2,4,5,6]) # 必要列抽出

# カラム名つける
df_seika = df_seika.rename(columns={
    9:"成果識別番号",
    1:"成果発生日時",
    2:"獲得方法",
    4:"メディア名",
    5:"LP名",
    6:"グロス(税込)"
})

# 日付と時間に列を分ける（レントラだけ仕様が違う）
ach_day  = df_seika['成果発生日時'].str.extract('(\d+[/-]\d+[/-]+\d+)', expand=True) # 成果発生日
ach_day  = ach_day.apply(lambda x: x.str.replace('-','/'))                           # "-"を"/"に変換
ach_time = df_seika['成果発生日時'].str.extract('(\d+:\d:*\d*)', expand=True)        # 成果発生時間
df_seika.insert(1,"成果発生日",ach_day)    # 成果発生日を追加
df_seika.insert(2,"成果発生時間",ach_time) # 成果発生時間を追加
df_seika = df_seika.drop("成果発生日時",axis=1) # 成果発生日時削除

# ASP名追加
df_seika.insert(5,"ASP","ブリーチ") 

In [77]:
import datetime

In [78]:
df = df_seika

In [80]:
# 今月のデータを抽出
# df = df[df['成果発生日'] <= datetime.date(2022,1,31)]
# df = df[df['成果発生日'] >= datetime.date(2022,1,1)]
datetime.date(2022,1,31) > datetime.date(2022,1,31)

False

In [64]:
df

In [74]:
print(datetime.date(2022,1,1))

2022-01-01


In [47]:
df

,成果識別番号,成果発生日,成果発生時間,獲得方法,メディア名,ASP,LP名,グロス(税込)
2,41577,2022/01/01,4:30,PPC,ブリーチ(PPC_2)ブリーチ_全身脱毛2ヵ月無料（両うで脱毛プレゼント）LP,ブリーチ,,"¥45,000"
3,41696,2022/01/01,22:32,AD,ブリーチ(AD_17)直フォーム,ブリーチ,,"¥40,000"
4,41741,2022/01/02,8:32,AD,ブリーチ(AD_17)直フォーム,ブリーチ,,"¥40,000"
5,41813,2022/01/02,16:51,PPC,ブリーチ(PPC_2)ブリーチ_全身脱毛2ヵ月無料（両うで脱毛プレゼント）LP,ブリーチ,,"¥45,000"
6,41828,2022/01/02,18:15,PPC,ブリーチ(PPC_2)ブリーチ_全身脱毛2ヵ月無料（両うで脱毛プレゼント）LP,ブリーチ,,"¥45,000"
7,41838,2022/01/02,18:42,PPC,ブリーチ(PPC_2)ブリーチ_全身脱毛2ヵ月無料（両うで脱毛プレゼント）LP,ブリーチ,,"¥45,000"
8,41847,2022/01/02,19:44,PPC,ブリーチ(PPC_2)ブリーチ_全身脱毛2ヵ月無料（両うで脱毛プレゼント）LP,ブリーチ,,"¥45,000"
9,41857,2022/01/02,21:11,PPC,ブリーチ(PPC_2)ブリーチ_全身脱毛2ヵ月無料（両うで脱毛プレゼント）LP,ブリーチ,,"¥45,000"
10,41880,2022/01/02,22:38,PPC,ブリーチ(PPC_2)ブリーチ_全身脱毛2ヵ月無料（両うで脱毛プレゼント）LP,ブリーチ,,"¥45,000"
11,41898,2022/01/03,0:23,AD,ブリーチ(AD_17)直フォーム,ブリーチ,,"¥40,000"
